# 125: ML Testing & Validation

## 🎯 Learning Objectives

By the end of this notebook, you will:
- **Understand** testing strategies specific to ML systems (beyond traditional software)
- **Implement** unit tests for data pipelines, feature engineering, and model components
- **Build** integration tests for end-to-end ML workflows
- **Apply** model validation frameworks to post-silicon yield prediction systems
- **Evaluate** test coverage and quality metrics for ML code
- **Design** property-based testing for ML invariants

## 📚 What is ML Testing & Validation?

**ML testing** extends traditional software testing to address the unique challenges of machine learning systems: non-deterministic behavior, data dependencies, gradual performance degradation, and the interplay between code, data, and models. Unlike traditional software where bugs are binary (works/doesn't work), ML bugs are often subtle performance regressions that require statistical testing and continuous monitoring.

**Why ML Testing is Different:**
- ✅ **Code + Data + Model**: Must test all three components and their interactions
- ✅ **Non-deterministic**: Same input can produce different outputs (stochastic algorithms)
- ✅ **Gradual degradation**: Performance slowly declines (data drift, concept drift)
- ✅ **No ground truth in production**: Can't verify correctness of predictions directly
- ✅ **Statistical testing**: Assertions use distributions, not exact values
- ✅ **Infrastructure dependencies**: Models depend on feature stores, data pipelines, serving systems

**Testing Pyramid for ML:**
1. **Unit Tests (70%)**: Data validation, feature engineering, model components
2. **Integration Tests (20%)**: End-to-end pipelines, model training, serving
3. **System Tests (10%)**: Performance, load testing, monitoring

## 🏭 Post-Silicon Validation Use Cases

**Yield Prediction Model Testing**
- Input: Test unit tests for feature engineering (Vdd normalization, power calculations)
- Tests: Assert feature ranges, null handling, statistical properties
- Output: Confidence that features are computed correctly before model training
- Value: Catch data quality issues before they corrupt model training (save 2-3 days retraining)

**Test Time Prediction Integration Tests**
- Input: End-to-end test from STDF data → features → model → prediction → API response
- Tests: Validate entire pipeline produces correct predictions within latency SLA
- Output: Ensure deployment doesn't break existing functionality
- Value: Prevent production incidents (99.9% uptime SLA for inline test binning)

**Binning Model Validation Framework**
- Input: New binning model candidate (v2.0) vs production model (v1.5)
- Tests: A/B comparison on holdout set, fairness checks, edge case testing
- Output: Statistical evidence v2.0 is better (or reject deployment)
- Value: Safe model updates, no accuracy regressions, compliance with test specifications

**Wafer Map Analysis Property Tests**
- Input: Spatial feature engineering functions (neighbor calculations, radial distance)
- Tests: Property-based tests (symmetry, boundary conditions, invariants)
- Output: Verify spatial logic correct for all wafer geometries
- Value: Catch edge cases (wafer edge, incomplete dies, coordinate systems)

## 🔄 ML Testing Workflow

```mermaid
graph TB
    subgraph "Development"
        A[Write Code] --> B[Unit Tests]
        B --> C[Local Validation]
    end
    
    subgraph "Data Pipeline Testing"
        D[Data Ingestion] --> E[Schema Validation]
        E --> F[Data Quality Tests]
        F --> G[Feature Engineering Tests]
    end
    
    subgraph "Model Testing"
        H[Model Training] --> I[Training Tests]
        I --> J[Model Validation]
        J --> K[Performance Tests]
    end
    
    subgraph "Integration Testing"
        L[End-to-End Pipeline] --> M[Integration Tests]
        M --> N[Load Tests]
        N --> O[Monitoring Tests]
    end
    
    subgraph "Production Validation"
        P[Deployment] --> Q[Smoke Tests]
        Q --> R[A/B Tests]
        R --> S[Shadow Mode]
    end
    
    C --> D
    G --> H
    K --> L
    O --> P
    S --> T[Production]
    
    style A fill:#e1f5ff
    style T fill:#e1ffe1
    style J fill:#fff5e1
    style M fill:#ffe1e1
```

## 📊 Learning Path Context

**Prerequisites:**
- **121_MLOps_Fundamentals.ipynb** - MLOps lifecycle, experiment tracking
- **122_MLflow_Complete_Guide.ipynb** - Model versioning
- **124_Feature_Store_Implementation.ipynb** - Feature engineering patterns

**Next Steps:**
- **126_Continuous_Training_Pipelines.ipynb** - Automated retraining with tests
- **127_Model_Serving_Patterns.ipynb** - Deployment with validation gates
- **128_ML_Monitoring.ipynb** - Production monitoring and alerting

---

Let's build robust ML testing frameworks! 🚀

In [ ]:
# Install ML testing libraries
# !pip install pytest pytest-cov hypothesis great-expectations scikit-learn pandas numpy

import pytest
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

print("ML testing libraries loaded")
print("Focus: Unit tests, integration tests, model validation")

In [ ]:
# Data validation tests using pytest

class TestSTDFDataSchema:
    """Unit tests for STDF data schema validation."""
    
    def test_required_columns_present(self):
        """Test that all required columns exist."""
        data = pd.DataFrame({
            'device_id': ['DEV001'],
            'vdd': [1.2],
            'idd': [250.0],
            'freq': [2.5],
            'temp': [85.0],
            'yield': [1]
        })
        
        required_cols = ['device_id', 'vdd', 'idd', 'freq', 'temp', 'yield']
        
        for col in required_cols:
            assert col in data.columns, f"Missing required column: {col}"
        
        print("✅ test_required_columns_present PASSED")
    
    def test_data_types_correct(self):
        """Test that columns have correct data types."""
        data = pd.DataFrame({
            'device_id': ['DEV001', 'DEV002'],
            'vdd': [1.2, 1.21],
            'idd': [250.0, 255.0],
            'freq': [2.5, 2.48],
            'temp': [85.0, 86.0],
            'yield': [1, 0]
        })
        
        assert data['device_id'].dtype == object, "device_id should be string"
        assert np.issubdtype(data['vdd'].dtype, np.floating), "vdd should be float"
        assert np.issubdtype(data['idd'].dtype, np.floating), "idd should be float"
        assert np.issubdtype(data['freq'].dtype, np.floating), "freq should be float"
        assert np.issubdtype(data['temp'].dtype, np.floating), "temp should be float"
        assert np.issubdtype(data['yield'].dtype, np.integer), "yield should be int"
        
        print("✅ test_data_types_correct PASSED")
    
    def test_parametric_ranges_valid(self):
        """Test that parametric values are within valid ranges."""
        data = pd.DataFrame({
            'device_id': ['DEV001'],
            'vdd': [1.2],
            'idd': [250.0],
            'freq': [2.5],
            'temp': [85.0],
            'yield': [1]
        })
        
        # Parametric limits (from test specifications)
        assert data['vdd'].between(0.9, 1.5).all(), "Vdd out of range (0.9-1.5V)"
        assert data['idd'].between(0, 500).all(), "Idd out of range (0-500mA)"
        assert data['freq'].between(1.0, 5.0).all(), "Freq out of range (1-5GHz)"
        assert data['temp'].between(-40, 125).all(), "Temp out of range (-40-125°C)"
        assert data['yield'].isin([0, 1]).all(), "Yield must be 0 or 1"
        
        print("✅ test_parametric_ranges_valid PASSED")
    
    def test_no_nulls_in_critical_columns(self):
        """Test that critical columns have no null values."""
        data = pd.DataFrame({
            'device_id': ['DEV001', 'DEV002'],
            'vdd': [1.2, 1.21],
            'idd': [250.0, 255.0],
            'freq': [2.5, 2.48],
            'temp': [85.0, 86.0],
            'yield': [1, 0]
        })
        
        critical_cols = ['device_id', 'vdd', 'idd', 'freq', 'yield']
        
        for col in critical_cols:
            null_count = data[col].isnull().sum()
            assert null_count == 0, f"Column {col} has {null_count} null values"
        
        print("✅ test_no_nulls_in_critical_columns PASSED")
    
    def test_device_id_unique(self):
        """Test that device IDs are unique (no duplicates)."""
        data = pd.DataFrame({
            'device_id': ['DEV001', 'DEV002', 'DEV003'],
            'vdd': [1.2, 1.21, 1.19],
            'idd': [250.0, 255.0, 248.0],
            'freq': [2.5, 2.48, 2.51],
            'temp': [85.0, 86.0, 84.0],
            'yield': [1, 0, 1]
        })
        
        assert data['device_id'].is_unique, "device_id column has duplicates"
        
        print("✅ test_device_id_unique PASSED")

# Run tests
tester = TestSTDFDataSchema()
tester.test_required_columns_present()
tester.test_data_types_correct()
tester.test_parametric_ranges_valid()
tester.test_no_nulls_in_critical_columns()
tester.test_device_id_unique()

print("\n✅ All data schema tests passed!")

In [ ]:
# Feature engineering functions (to be tested)

def compute_power(vdd, idd):
    """Calculate power consumption in milliwatts."""
    return vdd * idd

def compute_power_efficiency(freq, power):
    """Calculate frequency per unit power (GHz/W)."""
    if power == 0:
        return np.nan  # Avoid division by zero
    return freq / power

def normalize_zscore(values):
    """Normalize values to Z-scores."""
    mean = np.mean(values)
    std = np.std(values)
    if std == 0:
        return np.zeros_like(values)  # Constant values → all zeros
    return (values - mean) / std

def compute_rolling_mean(values, window=7):
    """Compute rolling mean with specified window."""
    if len(values) < window:
        return np.full_like(values, np.nan, dtype=float)
    return pd.Series(values).rolling(window=window, min_periods=1).mean().values

# Unit tests for feature engineering

class TestFeatureEngineering:
    """Unit tests for feature engineering functions."""
    
    def test_compute_power_normal_case(self):
        """Test power calculation with normal values."""
        vdd = 1.2  # Volts
        idd = 250.0  # mA
        expected_power = 300.0  # mW
        
        actual_power = compute_power(vdd, idd)
        
        assert np.isclose(actual_power, expected_power, rtol=1e-6), \
            f"Power calculation incorrect: expected {expected_power}, got {actual_power}"
        
        print("✅ test_compute_power_normal_case PASSED")
    
    def test_compute_power_edge_cases(self):
        """Test power calculation with edge cases."""
        # Zero current
        assert compute_power(1.2, 0) == 0, "Power should be 0 when current is 0"
        
        # Zero voltage
        assert compute_power(0, 250.0) == 0, "Power should be 0 when voltage is 0"
        
        # Both zero
        assert compute_power(0, 0) == 0, "Power should be 0 when both are 0"
        
        print("✅ test_compute_power_edge_cases PASSED")
    
    def test_power_efficiency_normal_case(self):
        """Test power efficiency calculation."""
        freq = 2.5  # GHz
        power = 300.0  # mW
        expected_efficiency = 2.5 / 300.0  # GHz/mW
        
        actual_efficiency = compute_power_efficiency(freq, power)
        
        assert np.isclose(actual_efficiency, expected_efficiency, rtol=1e-6), \
            f"Efficiency calculation incorrect"
        
        print("✅ test_power_efficiency_normal_case PASSED")
    
    def test_power_efficiency_zero_power(self):
        """Test power efficiency with zero power (division by zero)."""
        freq = 2.5
        power = 0
        
        result = compute_power_efficiency(freq, power)
        
        assert np.isnan(result), "Should return NaN when power is 0"
        
        print("✅ test_power_efficiency_zero_power PASSED")
    
    def test_normalize_zscore_normal_case(self):
        """Test Z-score normalization."""
        values = np.array([1.0, 2.0, 3.0, 4.0, 5.0])
        
        zscores = normalize_zscore(values)
        
        # Z-scores should have mean ≈ 0 and std ≈ 1
        assert np.isclose(np.mean(zscores), 0, atol=1e-6), "Z-scores mean should be 0"
        assert np.isclose(np.std(zscores), 1, atol=1e-6), "Z-scores std should be 1"
        
        print("✅ test_normalize_zscore_normal_case PASSED")
    
    def test_normalize_zscore_constant_values(self):
        """Test Z-score with constant values (zero std)."""
        values = np.array([5.0, 5.0, 5.0, 5.0])
        
        zscores = normalize_zscore(values)
        
        # All values same → all Z-scores should be 0
        assert np.all(zscores == 0), "Constant values should have Z-scores of 0"
        
        print("✅ test_normalize_zscore_constant_values PASSED")
    
    def test_rolling_mean_normal_case(self):
        """Test rolling mean calculation."""
        values = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
        window = 3
        
        rolling = compute_rolling_mean(values, window=window)
        
        # Check specific values
        assert np.isclose(rolling[2], 2.0, atol=1e-6), "Rolling mean at index 2 should be 2.0"
        assert np.isclose(rolling[9], 9.0, atol=1e-6), "Rolling mean at index 9 should be 9.0"
        
        print("✅ test_rolling_mean_normal_case PASSED")
    
    def test_rolling_mean_insufficient_data(self):
        """Test rolling mean with fewer values than window."""
        values = np.array([1, 2])
        window = 7
        
        rolling = compute_rolling_mean(values, window=window)
        
        # Should return NaN when data < window
        assert np.all(np.isnan(rolling)), "Should return NaN when data < window"
        
        print("✅ test_rolling_mean_insufficient_data PASSED")
    
    def test_deterministic_output(self):
        """Test that functions produce deterministic outputs."""
        vdd = 1.2
        idd = 250.0
        
        # Call multiple times
        result1 = compute_power(vdd, idd)
        result2 = compute_power(vdd, idd)
        result3 = compute_power(vdd, idd)
        
        assert result1 == result2 == result3, "Function should be deterministic"
        
        print("✅ test_deterministic_output PASSED")

# Run tests
fe_tester = TestFeatureEngineering()
fe_tester.test_compute_power_normal_case()
fe_tester.test_compute_power_edge_cases()
fe_tester.test_power_efficiency_normal_case()
fe_tester.test_power_efficiency_zero_power()
fe_tester.test_normalize_zscore_normal_case()
fe_tester.test_normalize_zscore_constant_values()
fe_tester.test_rolling_mean_normal_case()
fe_tester.test_rolling_mean_insufficient_data()
fe_tester.test_deterministic_output()

print("\n✅ All feature engineering tests passed!")

In [ ]:
# Property-based testing with Hypothesis
from hypothesis import given, strategies as st, assume

class TestFeatureProperties:
    """Property-based tests for feature engineering."""
    
    @given(
        vdd=st.floats(min_value=0.1, max_value=2.0, allow_nan=False, allow_infinity=False),
        idd=st.floats(min_value=0, max_value=1000, allow_nan=False, allow_infinity=False)
    )
    def test_power_always_non_negative(self, vdd, idd):
        """Property: Power should always be non-negative."""
        power = compute_power(vdd, idd)
        assert power >= 0, f"Power should be non-negative, got {power}"
    
    @given(
        vdd=st.floats(min_value=0.1, max_value=2.0, allow_nan=False, allow_infinity=False),
        idd=st.floats(min_value=0.1, max_value=1000, allow_nan=False, allow_infinity=False)
    )
    def test_power_increases_with_voltage(self, vdd, idd):
        """Property: Increasing voltage should increase power (current constant)."""
        power1 = compute_power(vdd, idd)
        power2 = compute_power(vdd * 1.1, idd)  # 10% voltage increase
        
        assert power2 > power1, "Power should increase with voltage"
    
    @given(
        values=st.lists(
            st.floats(min_value=-100, max_value=100, allow_nan=False, allow_infinity=False),
            min_size=10,
            max_size=100
        )
    )
    def test_zscore_mean_approximately_zero(self, values):
        """Property: Z-scores should have mean ≈ 0."""
        values_array = np.array(values)
        
        # Skip if all values are the same (would result in division by zero)
        if np.std(values_array) < 1e-10:
            return
        
        zscores = normalize_zscore(values_array)
        mean_zscore = np.mean(zscores)
        
        assert abs(mean_zscore) < 1e-6, f"Z-score mean should be ~0, got {mean_zscore}"
    
    @given(
        values=st.lists(
            st.floats(min_value=-100, max_value=100, allow_nan=False, allow_infinity=False),
            min_size=10,
            max_size=100
        )
    )
    def test_zscore_std_approximately_one(self, values):
        """Property: Z-scores should have std ≈ 1."""
        values_array = np.array(values)
        
        # Skip if all values are the same
        if np.std(values_array) < 1e-10:
            return
        
        zscores = normalize_zscore(values_array)
        std_zscore = np.std(zscores)
        
        assert abs(std_zscore - 1.0) < 1e-6, f"Z-score std should be ~1, got {std_zscore}"
    
    @given(
        freq=st.floats(min_value=0.1, max_value=5.0, allow_nan=False, allow_infinity=False),
        power=st.floats(min_value=0.1, max_value=1000, allow_nan=False, allow_infinity=False)
    )
    def test_efficiency_inverse_relationship(self, freq, power):
        """Property: Doubling power should halve efficiency (freq constant)."""
        eff1 = compute_power_efficiency(freq, power)
        eff2 = compute_power_efficiency(freq, power * 2)
        
        assert np.isclose(eff2, eff1 / 2, rtol=1e-6), \
            "Doubling power should halve efficiency"

# Run property-based tests
print("Running property-based tests (100 examples per test)...")
print("(Hypothesis will generate random inputs to find edge cases)\n")

prop_tester = TestFeatureProperties()

# Note: In real pytest, these would run automatically with @given decorator
# For notebook demonstration, we'll run them manually with specific examples

print("✅ test_power_always_non_negative:")
prop_tester.test_power_always_non_negative(1.2, 250.0)
prop_tester.test_power_always_non_negative(0.5, 100.0)
prop_tester.test_power_always_non_negative(1.8, 400.0)
print("   Verified for multiple random inputs")

print("\n✅ test_power_increases_with_voltage:")
prop_tester.test_power_increases_with_voltage(1.0, 200.0)
prop_tester.test_power_increases_with_voltage(1.5, 300.0)
print("   Verified voltage-power relationship")

print("\n✅ test_zscore_mean_approximately_zero:")
prop_tester.test_zscore_mean_approximately_zero([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
prop_tester.test_zscore_mean_approximately_zero([10.5, 11.2, 9.8, 10.1, 10.9])
print("   Verified Z-score mean property")

print("\n✅ test_zscore_std_approximately_one:")
prop_tester.test_zscore_std_approximately_one([1, 2, 3, 4, 5])
prop_tester.test_zscore_std_approximately_one([100, 110, 90, 95, 105])
print("   Verified Z-score std property")

print("\n✅ test_efficiency_inverse_relationship:")
prop_tester.test_efficiency_inverse_relationship(2.5, 300.0)
prop_tester.test_efficiency_inverse_relationship(3.0, 400.0)
print("   Verified efficiency-power relationship")

print("\n✅ All property-based tests passed!")
print("In production, Hypothesis would test 100+ random inputs automatically")

In [ ]:
# Generate synthetic training data
np.random.seed(42)

def generate_yield_data(n_samples=1000):
    """Generate synthetic device yield data for testing."""
    vdd = np.random.normal(1.2, 0.02, n_samples)
    idd = np.random.normal(250, 15, n_samples)
    freq = np.random.normal(2.5, 0.1, n_samples)
    temp = np.random.normal(85, 5, n_samples)
    
    # Yield based on parametric limits
    yield_label = (
        (vdd >= 1.15) & (vdd <= 1.25) &
        (idd <= 280) &
        (freq >= 2.3) &
        (temp <= 100)
    ).astype(int)
    
    X = pd.DataFrame({
        'vdd': vdd,
        'idd': idd,
        'freq': freq,
        'temp': temp
    })
    y = yield_label
    
    return X, y

# Model testing class

class TestYieldPredictionModel:
    """Tests for yield prediction model."""
    
    def test_model_trains_without_error(self):
        """Test that model trains successfully."""
        X, y = generate_yield_data(n_samples=500)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        model = RandomForestClassifier(n_estimators=10, random_state=42, max_depth=5)
        
        # Should not raise any exception
        model.fit(X_train, y_train)
        
        print("✅ test_model_trains_without_error PASSED")
    
    def test_model_makes_predictions(self):
        """Test that model produces predictions."""
        X, y = generate_yield_data(n_samples=500)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        model = RandomForestClassifier(n_estimators=10, random_state=42)
        model.fit(X_train, y_train)
        
        predictions = model.predict(X_test)
        
        # Check predictions have correct shape
        assert len(predictions) == len(X_test), "Prediction count mismatch"
        
        # Check predictions are valid labels (0 or 1)
        assert set(predictions).issubset({0, 1}), "Invalid prediction labels"
        
        print("✅ test_model_makes_predictions PASSED")
    
    def test_model_predicts_probabilities(self):
        """Test that model produces valid probabilities."""
        X, y = generate_yield_data(n_samples=500)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        model = RandomForestClassifier(n_estimators=10, random_state=42)
        model.fit(X_train, y_train)
        
        probabilities = model.predict_proba(X_test)
        
        # Check shape (n_samples, n_classes)
        assert probabilities.shape == (len(X_test), 2), "Probability shape incorrect"
        
        # Check probabilities sum to 1
        prob_sums = probabilities.sum(axis=1)
        assert np.allclose(prob_sums, 1.0), "Probabilities don't sum to 1"
        
        # Check probabilities in [0, 1]
        assert np.all(probabilities >= 0) and np.all(probabilities <= 1), \
            "Probabilities out of [0, 1] range"
        
        print("✅ test_model_predicts_probabilities PASSED")
    
    def test_model_beats_baseline(self):
        """Test that model performs better than random guessing."""
        X, y = generate_yield_data(n_samples=500)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        model = RandomForestClassifier(n_estimators=50, random_state=42)
        model.fit(X_train, y_train)
        
        accuracy = model.score(X_test, y_test)
        
        # Model should beat random guessing (50% for binary)
        baseline_accuracy = 0.5
        assert accuracy > baseline_accuracy + 0.1, \
            f"Model accuracy ({accuracy:.3f}) not significantly better than baseline ({baseline_accuracy})"
        
        print(f"✅ test_model_beats_baseline PASSED (accuracy: {accuracy:.3f} > {baseline_accuracy + 0.1})")
    
    def test_model_reproducibility(self):
        """Test that model produces same results with same seed."""
        X, y = generate_yield_data(n_samples=500)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Train two models with same seed
        model1 = RandomForestClassifier(n_estimators=10, random_state=42)
        model1.fit(X_train, y_train)
        pred1 = model1.predict(X_test)
        
        model2 = RandomForestClassifier(n_estimators=10, random_state=42)
        model2.fit(X_train, y_train)
        pred2 = model2.predict(X_test)
        
        # Predictions should be identical
        assert np.array_equal(pred1, pred2), "Model not reproducible with same seed"
        
        print("✅ test_model_reproducibility PASSED")
    
    def test_model_not_predicting_all_same_class(self):
        """Test that model doesn't predict all zeros or all ones."""
        X, y = generate_yield_data(n_samples=500)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        model = RandomForestClassifier(n_estimators=50, random_state=42)
        model.fit(X_train, y_train)
        
        predictions = model.predict(X_test)
        
        # Check both classes are predicted
        unique_predictions = set(predictions)
        assert len(unique_predictions) > 1, "Model predicting only one class (broken model)"
        
        print(f"✅ test_model_not_predicting_all_same_class PASSED (predicts {unique_predictions})")
    
    def test_model_feature_importance_not_zero(self):
        """Test that model uses features (importance > 0)."""
        X, y = generate_yield_data(n_samples=500)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        model = RandomForestClassifier(n_estimators=50, random_state=42)
        model.fit(X_train, y_train)
        
        importances = model.feature_importances_
        
        # At least one feature should have non-zero importance
        assert np.any(importances > 0), "All feature importances are zero (model not learning)"
        
        # Sum of importances should be 1.0
        assert np.isclose(importances.sum(), 1.0), "Feature importances don't sum to 1"
        
        print(f"✅ test_model_feature_importance_not_zero PASSED")

# Run model tests
model_tester = TestYieldPredictionModel()
model_tester.test_model_trains_without_error()
model_tester.test_model_makes_predictions()
model_tester.test_model_predicts_probabilities()
model_tester.test_model_beats_baseline()
model_tester.test_model_reproducibility()
model_tester.test_model_not_predicting_all_same_class()
model_tester.test_model_feature_importance_not_zero()

print("\n✅ All model tests passed!")

In [ ]:
# Integration tests for ML pipeline

class YieldPredictionPipeline:
    """Complete yield prediction pipeline for testing."""
    
    def __init__(self):
        self.model = None
        self.feature_columns = ['vdd', 'idd', 'freq', 'temp', 'power']
    
    def train(self, data):
        """Train pipeline on data."""
        # Extract features
        X = data[['vdd', 'idd', 'freq', 'temp']].copy()
        X['power'] = compute_power(X['vdd'], X['idd'])
        y = data['yield']
        
        # Train model
        self.model = RandomForestClassifier(n_estimators=50, random_state=42)
        self.model.fit(X, y)
        
        return self
    
    def predict(self, data):
        """Make predictions on new data."""
        if self.model is None:
            raise ValueError("Pipeline not trained")
        
        # Extract and engineer features
        X = data[['vdd', 'idd', 'freq', 'temp']].copy()
        X['power'] = compute_power(X['vdd'], X['idd'])
        
        # Predict
        predictions = self.model.predict(X)
        probabilities = self.model.predict_proba(X)
        
        return predictions, probabilities

class TestMLPipelineIntegration:
    """Integration tests for complete ML pipeline."""
    
    def test_end_to_end_pipeline(self):
        """Test complete pipeline: data → features → train → predict."""
        # Generate training data
        X, y = generate_yield_data(n_samples=1000)
        train_data = X.copy()
        train_data['yield'] = y
        
        # Train pipeline
        pipeline = YieldPredictionPipeline()
        pipeline.train(train_data)
        
        # Generate test data
        X_test, y_test = generate_yield_data(n_samples=200)
        test_data = X_test.copy()
        
        # Predict
        predictions, probabilities = pipeline.predict(test_data)
        
        # Verify outputs
        assert len(predictions) == 200, "Wrong number of predictions"
        assert probabilities.shape == (200, 2), "Wrong probability shape"
        assert set(predictions).issubset({0, 1}), "Invalid predictions"
        
        # Verify accuracy
        accuracy = accuracy_score(y_test, predictions)
        assert accuracy > 0.6, f"Pipeline accuracy too low: {accuracy:.3f}"
        
        print(f"✅ test_end_to_end_pipeline PASSED (accuracy: {accuracy:.3f})")
    
    def test_pipeline_handles_single_sample(self):
        """Test pipeline can process single device."""
        # Train pipeline
        X, y = generate_yield_data(n_samples=1000)
        train_data = X.copy()
        train_data['yield'] = y
        
        pipeline = YieldPredictionPipeline()
        pipeline.train(train_data)
        
        # Single device
        single_device = pd.DataFrame({
            'vdd': [1.2],
            'idd': [250.0],
            'freq': [2.5],
            'temp': [85.0]
        })
        
        predictions, probabilities = pipeline.predict(single_device)
        
        assert len(predictions) == 1, "Should predict for single device"
        assert predictions[0] in [0, 1], "Invalid prediction"
        
        print("✅ test_pipeline_handles_single_sample PASSED")
    
    def test_pipeline_handles_batch(self):
        """Test pipeline can process large batches efficiently."""
        import time
        
        # Train pipeline
        X, y = generate_yield_data(n_samples=1000)
        train_data = X.copy()
        train_data['yield'] = y
        
        pipeline = YieldPredictionPipeline()
        pipeline.train(train_data)
        
        # Large batch
        X_batch, _ = generate_yield_data(n_samples=10000)
        
        start_time = time.time()
        predictions, _ = pipeline.predict(X_batch)
        elapsed_ms = (time.time() - start_time) * 1000
        
        # Check latency (should be <500ms for 10K predictions)
        assert elapsed_ms < 500, f"Batch prediction too slow: {elapsed_ms:.1f}ms"
        assert len(predictions) == 10000, "Wrong prediction count"
        
        throughput = 10000 / (elapsed_ms / 1000)
        print(f"✅ test_pipeline_handles_batch PASSED (latency: {elapsed_ms:.1f}ms, throughput: {throughput:.0f} pred/sec)")
    
    def test_pipeline_error_handling(self):
        """Test pipeline handles invalid inputs gracefully."""
        # Train pipeline
        X, y = generate_yield_data(n_samples=1000)
        train_data = X.copy()
        train_data['yield'] = y
        
        pipeline = YieldPredictionPipeline()
        pipeline.train(train_data)
        
        # Missing column
        invalid_data = pd.DataFrame({
            'vdd': [1.2],
            'idd': [250.0]
            # Missing freq, temp
        })
        
        try:
            pipeline.predict(invalid_data)
            assert False, "Should raise error on missing columns"
        except KeyError:
            print("✅ test_pipeline_error_handling PASSED (correctly raises error)")
    
    def test_pipeline_reproducibility(self):
        """Test pipeline produces same results on same data."""
        # Train pipeline twice
        X, y = generate_yield_data(n_samples=1000)
        train_data = X.copy()
        train_data['yield'] = y
        
        pipeline1 = YieldPredictionPipeline()
        pipeline1.train(train_data)
        
        pipeline2 = YieldPredictionPipeline()
        pipeline2.train(train_data)
        
        # Test data
        X_test, _ = generate_yield_data(n_samples=100)
        
        pred1, _ = pipeline1.predict(X_test)
        pred2, _ = pipeline2.predict(X_test)
        
        # Predictions should be identical (same random seed)
        assert np.array_equal(pred1, pred2), "Pipeline not reproducible"
        
        print("✅ test_pipeline_reproducibility PASSED")

# Run integration tests
integration_tester = TestMLPipelineIntegration()
integration_tester.test_end_to_end_pipeline()
integration_tester.test_pipeline_handles_single_sample()
integration_tester.test_pipeline_handles_batch()
integration_tester.test_pipeline_error_handling()
integration_tester.test_pipeline_reproducibility()

print("\n✅ All integration tests passed!")

## 7. Real-World Project Templates

**8 production-ready ML testing projects** (4 post-silicon + 4 general AI/ML).

Each project includes:
- Complete test strategy
- Test coverage targets
- Critical tests to implement
- Business value and impact

In [ ]:
projects = """
================================================================================
REAL-WORLD ML TESTING PROJECTS
================================================================================

POST-SILICON VALIDATION PROJECTS
---------------------------------

1. YIELD PREDICTION MODEL TEST SUITE (COMPREHENSIVE)
   
   Objective: 90% test coverage for yield prediction pipeline, catch bugs
              before production deployment
   
   Success Metrics:
   - Code coverage: >90% (pytest-cov)
   - Test count: 100+ unit tests, 20+ integration tests
   - CI/CD: All tests pass before merge
   - Bug detection: Catch 95% of bugs pre-production
   
   Test Strategy:
   
   **Unit Tests (70 tests):**
   - Data validation: Schema, ranges, nulls (15 tests)
   - Feature engineering: Power, efficiency, Z-scores (20 tests)
   - Model components: Training, prediction, probabilities (15 tests)
   - Utilities: Data loading, preprocessing, metrics (20 tests)
   
   **Integration Tests (15 tests):**
   - End-to-end pipeline: STDF → features → model → prediction
   - Performance: Latency <100ms p99, throughput >1000 pred/sec
   - Error handling: Graceful degradation on bad inputs
   - Reproducibility: Same seed → same results
   
   **Property-Based Tests (10 tests):**
   - Feature invariants: Power ≥ 0, Z-score mean ≈ 0
   - Model properties: Predictions in [0,1], probabilities sum to 1
   - Edge cases: Extreme values, boundary conditions
   
   **System Tests (5 tests):**
   - Load testing: 10K concurrent predictions
   - Monitoring: Alerts trigger on drift
   - A/B testing: New model vs production model
   
   Critical Tests:
   ```python
   def test_yield_prediction_accuracy_above_threshold():
       \"\"\"CRITICAL: Model must achieve >85% accuracy.\"\"\"
       model = train_yield_model(data)
       accuracy = evaluate_model(model, test_data)
       assert accuracy > 0.85, f"Accuracy {accuracy:.3f} below threshold"
   
   def test_no_data_leakage():
       \"\"\"CRITICAL: Features use only past data (point-in-time).\"\"\"
       features = compute_features(data, timestamp=T)
       future_data = data[data['timestamp'] > T]
       assert not feature_uses_future_data(features, future_data)
   
   def test_prediction_latency_sla():
       \"\"\"CRITICAL: Predictions must complete <100ms p99.\"\"\"
       latencies = benchmark_prediction_latency(model, n=10000)
       p99_latency = np.percentile(latencies, 99)
       assert p99_latency < 100, f"p99 latency {p99_latency}ms exceeds 100ms SLA"
   ```
   
   Business Value: Prevent production incidents ($500K/incident), faster
                   development (CI catches bugs early), compliance (audit trail)


2. TEST TIME PREDICTION MODEL VALIDATION FRAMEWORK
   
   Objective: Validate test time prediction model before deployment, ensure
              <5% MAPE and no catastrophic failures
   
   Success Metrics:
   - MAPE: <5% on holdout set
   - No predictions >10x actual test time (catastrophic errors)
   - Passes 30+ validation checks
   - Fairness: No bias across device types
   
   Validation Framework:
   
   **Performance Validation:**
   - Accuracy: MAPE, RMSE, R² score
   - Error distribution: Check for systematic bias
   - Percentile accuracy: P10, P50, P90 predictions
   
   **Robustness Validation:**
   - Edge cases: Fast devices (<1 sec), slow devices (>60 sec)
   - Outliers: Handle 3-sigma extreme test times
   - Missing features: Graceful degradation with imputation
   
   **Fairness Validation:**
   - Device type parity: Equal accuracy across types
   - Lot-to-lot consistency: No bias toward specific lots
   - Wafer position: No edge vs center bias
   
   **Safety Validation:**
   - No catastrophic errors: Predictions within 10x actual
   - Conservative estimates: Better to overestimate than underestimate
   - Alarm on outliers: Flag predictions >3 std from mean
   
   Implementation:
   ```python
   class TestTimeModelValidator:
       def __init__(self, model, test_data):
           self.model = model
           self.test_data = test_data
       
       def validate_accuracy(self):
           predictions = self.model.predict(self.test_data.X)
           actual = self.test_data.y
           
           mape = mean_absolute_percentage_error(actual, predictions)
           assert mape < 0.05, f"MAPE {mape:.3f} exceeds 5% threshold"
       
       def validate_no_catastrophic_errors(self):
           predictions = self.model.predict(self.test_data.X)
           actual = self.test_data.y
           
           max_error = (predictions / actual).max()
           assert max_error < 10, f"Catastrophic error: {max_error}x actual"
       
       def validate_fairness(self):
           for device_type in self.test_data['device_type'].unique():
               subset = self.test_data[self.test_data['device_type'] == device_type]
               accuracy = self.model.score(subset.X, subset.y)
               assert accuracy > 0.80, f"{device_type} accuracy too low"
   ```
   
   Business Value: $10M annual savings (optimize test flow), prevent
                   underestimating test time (capacity planning), compliance


3. WAFER MAP SPATIAL FEATURE TESTING (EDGE CASES)
   
   Objective: Exhaustive testing of spatial feature engineering for wafer maps,
              catch coordinate system bugs, edge effects, incomplete wafers
   
   Success Metrics:
   - Property tests: 1000+ random wafer geometries tested
   - Edge coverage: All wafer edge positions tested
   - Symmetry verification: Radial features symmetric
   - Coordinate systems: Cartesian and polar both correct
   
   Spatial Property Tests:
   
   **Neighbor Invariants:**
   ```python
   @given(die_x=st.integers(0, 50), die_y=st.integers(0, 50))
   def test_neighbor_count_max_eight(die_x, die_y):
       \"\"\"Property: Die can have at most 8 neighbors.\"\"\"
       wafer = generate_test_wafer()
       neighbors = get_neighbors(wafer, die_x, die_y)
       assert len(neighbors) <= 8
   
   @given(die_x=st.integers(0, 50), die_y=st.integers(0, 50))
   def test_neighbor_symmetry(die_x, die_y):
       \"\"\"Property: If A is neighbor of B, B is neighbor of A.\"\"\"
       wafer = generate_test_wafer()
       neighbors_of_A = get_neighbors(wafer, die_x, die_y)
       for neighbor in neighbors_of_A:
           neighbors_of_neighbor = get_neighbors(wafer, neighbor.x, neighbor.y)
           assert (die_x, die_y) in [(n.x, n.y) for n in neighbors_of_neighbor]
   ```
   
   **Radial Distance Invariants:**
   ```python
   def test_radial_distance_center_zero():
       \"\"\"Center die has radial distance 0.\"\"\"
       wafer = generate_test_wafer(radius=50)
       center_distance = compute_radial_distance(25, 25, wafer.center)
       assert center_distance == 0
   
   def test_radial_distance_increases_outward():
       \"\"\"Distance increases moving away from center.\"\"\"
       wafer = generate_test_wafer()
       for r in range(0, 50):
           distance = compute_radial_distance(25 + r, 25, wafer.center)
           next_distance = compute_radial_distance(25 + r + 1, 25, wafer.center)
           assert next_distance > distance
   ```
   
   **Edge Effect Tests:**
   ```python
   def test_edge_dies_flagged():
       \"\"\"Dies near wafer edge should be flagged.\"\"\"
       wafer = generate_test_wafer(radius=50)
       edge_distance_threshold = 5  # mm from edge
       
       for die in wafer.dies:
           dist_to_edge = compute_distance_to_edge(die, wafer.radius)
           is_edge = is_edge_die(die, wafer.radius, threshold=edge_distance_threshold)
           
           if dist_to_edge < edge_distance_threshold:
               assert is_edge, f"Die at {die.x},{die.y} should be flagged as edge"
           else:
               assert not is_edge
   ```
   
   Business Value: Prevent spatial analysis errors (5% yield improvement = $10M),
                   correct wafer map visualizations, accurate defect clustering


4. BINNING MODEL A/B TEST FRAMEWORK
   
   Objective: Statistical testing framework to validate new binning model (v2.0)
              vs production (v1.5) before deployment
   
   Success Metrics:
   - Statistical significance: p-value <0.05 for accuracy improvement
   - Practical significance: +2% F1 score improvement
   - No regression: All bins maintain or improve accuracy
   - Consistency: Results stable across 5 different random seeds
   
   A/B Test Framework:
   
   **Statistical Tests:**
   ```python
   class BinningModelABTest:
       def __init__(self, model_v1, model_v2, test_data):
           self.model_v1 = model_v1
           self.model_v2 = model_v2
           self.test_data = test_data
       
       def test_accuracy_improvement_significant(self):
           \"\"\"Test if v2.0 accuracy is statistically better than v1.5.\"\"\"
           from scipy.stats import wilcoxon
           
           # Bootstrap 100 times
           v1_accuracies = []
           v2_accuracies = []
           
           for seed in range(100):
               sample = self.test_data.sample(frac=0.8, random_state=seed)
               v1_acc = accuracy_score(sample.y, self.model_v1.predict(sample.X))
               v2_acc = accuracy_score(sample.y, self.model_v2.predict(sample.X))
               v1_accuracies.append(v1_acc)
               v2_accuracies.append(v2_acc)
           
           # Wilcoxon signed-rank test (paired samples)
           statistic, p_value = wilcoxon(v2_accuracies, v1_accuracies, alternative='greater')
           
           assert p_value < 0.05, f"v2.0 not significantly better (p={p_value:.4f})"
           
           mean_improvement = np.mean(v2_accuracies) - np.mean(v1_accuracies)
           assert mean_improvement > 0.02, f"Improvement too small: {mean_improvement:.3f}"
       
       def test_no_bin_regression(self):
           \"\"\"Test that no bin category has worse accuracy in v2.0.\"\"\"
           for bin_category in self.test_data['bin'].unique():
               subset = self.test_data[self.test_data['bin'] == bin_category]
               
               v1_f1 = f1_score(subset.y, self.model_v1.predict(subset.X))
               v2_f1 = f1_score(subset.y, self.model_v2.predict(subset.X))
               
               assert v2_f1 >= v1_f1 - 0.01, \
                   f"Bin {bin_category} regressed: v1={v1_f1:.3f} → v2={v2_f1:.3f}"
   ```
   
   **Shadow Mode Testing:**
   - Run v2.0 in shadow (doesn't affect production)
   - Compare predictions to v1.5 on live traffic
   - Collect 7 days of data before go-live
   
   Business Value: Safe model updates (no production regressions), data-driven
                   decisions (statistical evidence), compliance (validation docs)


GENERAL AI/ML PROJECTS
----------------------

5. FRAUD DETECTION MODEL TEST SUITE
   
   Objective: Comprehensive testing for fraud detection model with focus on
              precision (minimize false positives) and recall (catch fraud)
   
   Success Metrics:
   - Precision: >95% (low false positive rate)
   - Recall: >90% (catch most fraud)
   - Test coverage: >85% code coverage
   - Response time: <50ms p99 (real-time blocking)
   
   Critical Tests:
   - Known fraud patterns: Model catches 100% of known fraud types
   - False positive rate: <5% on legitimate transactions
   - Edge cases: High-value transactions, cross-border, new merchants
   - Adversarial robustness: Resistant to evasion techniques
   - Fairness: No demographic bias
   
   Business Value: $50M annual fraud prevented, <2% customer friction


6. CHURN PREDICTION MODEL VALIDATION
   
   Objective: Validate churn prediction model with emphasis on early detection
              (30 days before churn) and actionable predictions
   
   Success Metrics:
   - AUC-ROC: >0.85 (discrimination)
   - Early detection: Predict 30 days ahead with >80% recall
   - Calibration: Predicted probabilities match actual churn rates
   - Actionable: Top 10% highest risk = 60%+ actual churn
   
   Validation Tests:
   - Temporal validation: Train on 2023, test on 2024 data
   - Cohort analysis: Consistent performance across customer segments
   - Calibration plots: Predicted vs actual churn rates aligned
   - Feature drift monitoring: Alert if distributions change
   
   Business Value: $100M customer retention, targeted interventions


7. RECOMMENDATION SYSTEM TESTING (E-COMMERCE)
   
   Objective: Test recommendation quality, diversity, and business metrics
              before A/B testing with live traffic
   
   Success Metrics:
   - Offline metrics: NDCG@10 >0.7, Precision@10 >0.4
   - Diversity: >80% recommendations cover >20 different categories
   - Novelty: >30% recommendations are new items (not previously purchased)
   - Fairness: All merchants get proportional exposure
   
   Test Types:
   - Offline replay: Simulate recommendations on historical data
   - User journey testing: Multi-step sessions, cart additions
   - Cold start testing: New users, new products
   - Business rule validation: Never recommend out-of-stock items
   
   Business Value: +15% CTR ($20M revenue), improved customer experience


8. DEMAND FORECASTING MODEL VALIDATION (RETAIL)
   
   Objective: Validate SKU-level demand forecasts with emphasis on inventory
              optimization and stockout prevention
   
   Success Metrics:
   - MAPE: <15% across all SKUs
   - Stockout prevention: >95% in-stock for high-demand items
   - Overstock minimization: <10% excess inventory
   - Forecast horizon: 28-day accuracy within 20%
   
   Validation Framework:
   - Backtesting: Rolling window validation on 2 years historical data
   - Promotion testing: Accuracy during promotional periods
   - Seasonality: Correct holiday and seasonal patterns
   - External factors: Weather, economic indicators correlation
   
   Business Value: $30M inventory reduction, 10% revenue increase


================================================================================
IMPLEMENTATION CHECKLIST (FOR ALL PROJECTS)
================================================================================

Phase 1: Test Infrastructure Setup (Week 1)
□ pytest framework configured
□ Code coverage reporting (pytest-cov)
□ CI/CD integration (GitHub Actions, Jenkins)
□ Test data fixtures (synthetic + real samples)
□ Mocking frameworks for external dependencies

Phase 2: Unit Test Implementation (Week 2-3)
□ Data validation tests (schema, ranges, nulls)
□ Feature engineering tests (correctness, edge cases)
□ Model component tests (training, prediction)
□ Utility function tests (helpers, metrics)
□ Target: >80% code coverage

Phase 3: Integration Test Implementation (Week 4)
□ End-to-end pipeline tests
□ Performance/latency tests
□ Error handling tests
□ Reproducibility tests
□ Load testing (concurrent requests)

Phase 4: Property-Based Testing (Week 5)
□ Hypothesis setup and configuration
□ Feature invariants (mathematical properties)
□ Model properties (output constraints)
□ Edge case generation (boundary conditions)
□ Run 1000+ random test cases per property

Phase 5: Model Validation (Week 6)
□ Accuracy/performance benchmarks
□ Fairness and bias testing
□ Robustness testing (adversarial, noise)
□ A/B testing framework
□ Shadow mode deployment

Phase 6: Production Monitoring Tests (Week 7)
□ Data drift detection tests
□ Model performance monitoring
□ Alert verification tests
□ Dashboard functionality tests
□ Incident response testing

Best Practices:
- Write tests BEFORE implementing features (TDD)
- Aim for 90% code coverage, 100% critical path coverage
- Run fast tests in CI (<5 min), slow tests nightly
- Mock external dependencies (databases, APIs)
- Use fixtures for common test data
- Property tests for mathematical invariants
- Integration tests for critical workflows
- Version test data with code
"""

print(projects)

In [ ]:
takeaways = """
================================================================================
KEY TAKEAWAYS: ML TESTING & VALIDATION
================================================================================

1. WHY ML TESTING IS DIFFERENT
   ---------------------------
   
   Traditional Software Testing:
   - Binary outcomes: Works or doesn't work
   - Deterministic: Same input → same output
   - Code only: Test logic and algorithms
   - Bugs are obvious: Crashes, wrong outputs
   
   ML System Testing:
   - Continuous outcomes: Accuracy 85% vs 90%
   - Non-deterministic: Randomness, stochastic algorithms
   - Code + Data + Model: Test all three components
   - Bugs are subtle: Gradual performance degradation
   
   Unique ML Challenges:
   1. **Data Dependencies**: Model performance depends on data quality
   2. **No Ground Truth**: Can't verify correctness in production
   3. **Gradual Degradation**: Data drift slowly reduces accuracy
   4. **Statistical Testing**: Assertions use distributions not exact values
   5. **Training-Serving Skew**: Features computed differently
   
   Testing Philosophy:
   - Unit tests: Test components in isolation
   - Integration tests: Test components together
   - System tests: Test entire ML system
   - Monitoring tests: Test production behavior
   
   ML Testing Pyramid:
   70% Unit Tests (fast, many)
   20% Integration Tests (medium speed)
   10% System Tests (slow, few)


2. UNIT TESTING DATA PIPELINES
   ----------------------------
   
   What to Test:
   
   **Schema Validation:**
   - Column names and data types correct
   - Required columns present
   - No unexpected columns
   
   ```python
   def test_schema():
       data = load_data()
       assert 'device_id' in data.columns
       assert data['vdd'].dtype == float
       assert len(data.columns) == 6  # Expected column count
   ```
   
   **Data Quality:**
   - Valid ranges (Vdd: 0.9-1.5V, Temp: -40 to 125°C)
   - Null handling (critical columns have no nulls)
   - Uniqueness constraints (device_id is unique)
   - Referential integrity (foreign keys exist)
   
   ```python
   def test_parametric_ranges():
       data = load_data()
       assert data['vdd'].between(0.9, 1.5).all()
       assert data['temp'].between(-40, 125).all()
       assert data['device_id'].is_unique
   ```
   
   **Statistical Properties:**
   - Distributions match expected (mean, std within bounds)
   - No extreme outliers (>5 sigma)
   - Correlations as expected (Vdd vs Idd positive)
   
   ```python
   def test_vdd_distribution():
       data = load_data()
       mean = data['vdd'].mean()
       std = data['vdd'].std()
       assert 1.15 < mean < 1.25, "Vdd mean out of expected range"
       assert std < 0.05, "Vdd std too high"
   ```
   
   **Data Freshness:**
   - Data timestamp within acceptable window
   - No stale data (>24 hours old)
   - Complete data (all expected batches present)
   
   Tools:
   - pytest: Unit testing framework
   - Great Expectations: Data validation library
   - Pandera: DataFrame schema validation
   - Hypothesis: Property-based testing


3. UNIT TESTING FEATURE ENGINEERING
   ---------------------------------
   
   Critical Tests:
   
   **Mathematical Correctness:**
   - Formulas implemented correctly (power = vdd * idd)
   - Units consistent (voltage in V, current in mA)
   - Precision adequate (floating point errors acceptable)
   
   ```python
   def test_power_calculation():
       vdd = 1.2  # V
       idd = 250.0  # mA
       power = compute_power(vdd, idd)
       assert np.isclose(power, 300.0, rtol=1e-6)  # 300 mW
   ```
   
   **Edge Cases:**
   - Zero values (power when current=0)
   - Negative values (handle or reject)
   - Infinities and NaNs (division by zero)
   - Empty arrays (min size checks)
   
   ```python
   def test_power_efficiency_zero_power():
       freq = 2.5
       power = 0
       efficiency = compute_power_efficiency(freq, power)
       assert np.isnan(efficiency), "Should return NaN for zero power"
   ```
   
   **Determinism:**
   - Same input → same output (reproducibility)
   - No random seeds in feature engineering
   - No current_time() calls (breaks reproducibility)
   
   ```python
   def test_deterministic_features():
       data = load_test_data()
       features1 = compute_features(data)
       features2 = compute_features(data)
       assert features1.equals(features2), "Features not deterministic"
   ```
   
   **Null Handling:**
   - Consistent strategy (impute with mean, median, or forward fill)
   - No silent NaN propagation
   - Document null handling policy
   
   ```python
   def test_null_handling():
       data = pd.DataFrame({'vdd': [1.2, np.nan, 1.21]})
       features = compute_features(data)
       assert features['vdd'].isnull().sum() == 0, "Nulls not handled"
   ```
   
   Best Practices:
   - Test happy path AND edge cases
   - Use parametrized tests for multiple inputs
   - Test both valid and invalid inputs
   - Verify error messages are clear


4. PROPERTY-BASED TESTING WITH HYPOTHESIS
   ----------------------------------------
   
   What is Property-Based Testing?
   Instead of writing specific test cases, define properties that should
   ALWAYS hold, then let Hypothesis generate 100+ random test cases.
   
   Example:
   ```python
   from hypothesis import given, strategies as st
   
   @given(
       vdd=st.floats(min_value=0.1, max_value=2.0),
       idd=st.floats(min_value=0, max_value=1000)
   )
   def test_power_always_non_negative(vdd, idd):
       power = compute_power(vdd, idd)
       assert power >= 0  # Property: Power is always non-negative
   ```
   
   Benefits:
   - Finds edge cases you didn't think of
   - Tests 100+ random inputs automatically
   - Minimal code (define property, not test cases)
   - Reproduces failures (shrinks to minimal failing case)
   
   Common Properties to Test:
   
   **Mathematical Invariants:**
   - Power ≥ 0 (always non-negative)
   - Probabilities sum to 1
   - Z-scores have mean ≈ 0, std ≈ 1
   
   **Relationships:**
   - Increasing voltage → increasing power (current constant)
   - Doubling power → halving efficiency (freq constant)
   - Symmetry: distance(A, B) = distance(B, A)
   
   **Boundary Conditions:**
   - Empty input → empty output (or error)
   - Single element → valid result
   - Large input (10K elements) → no crash
   
   **Inverse Functions:**
   - encode(decode(x)) = x
   - normalize → denormalize returns original
   
   When to Use:
   - Mathematical functions (easy to define properties)
   - Data transformations (invariants should hold)
   - Parsers/serializers (round-trip property)
   - NOT for complex ML models (hard to define properties)


5. MODEL TESTING
   --------------
   
   Training Tests:
   
   **Smoke Test:**
   - Model trains without errors
   - Training completes in reasonable time (<5 min for small dataset)
   - Model converges (loss decreases)
   
   ```python
   def test_model_trains():
       X, y = load_train_data()
       model = RandomForestClassifier()
       model.fit(X, y)  # Should not raise exception
   ```
   
   **Learning Test:**
   - Model performs better than random guessing
   - Model performs better than simple baseline (mean predictor)
   - Training accuracy > test accuracy (some overfitting expected)
   
   ```python
   def test_model_beats_baseline():
       X_train, X_test, y_train, y_test = split_data()
       model.fit(X_train, y_train)
       accuracy = model.score(X_test, y_test)
       baseline = 0.5  # Random guessing for binary classification
       assert accuracy > baseline + 0.1
   ```
   
   Prediction Tests:
   
   **Output Shape:**
   - Correct number of predictions (matches input size)
   - Correct dimensionality (1D array for binary, 2D for multi-class)
   
   **Output Validity:**
   - Predictions are valid labels (0, 1 for binary)
   - Probabilities in [0, 1] range
   - Probabilities sum to 1 across classes
   
   ```python
   def test_prediction_probabilities_valid():
       model.fit(X_train, y_train)
       probs = model.predict_proba(X_test)
       assert (probs >= 0).all() and (probs <= 1).all()
       assert np.allclose(probs.sum(axis=1), 1.0)
   ```
   
   **Prediction Diversity:**
   - Model doesn't predict all zeros or all ones
   - Predicted classes match actual class distribution (roughly)
   
   ```python
   def test_model_predicts_both_classes():
       predictions = model.predict(X_test)
       unique_preds = set(predictions)
       assert len(unique_preds) > 1, "Model predicts only one class"
   ```
   
   **Reproducibility:**
   - Same random seed → same results
   - Model serialization → deserialization preserves behavior
   
   ```python
   def test_model_reproducible():
       model1 = RandomForestClassifier(random_state=42)
       model1.fit(X_train, y_train)
       pred1 = model1.predict(X_test)
       
       model2 = RandomForestClassifier(random_state=42)
       model2.fit(X_train, y_train)
       pred2 = model2.predict(X_test)
       
       assert np.array_equal(pred1, pred2)
   ```


6. INTEGRATION TESTING
   --------------------
   
   What to Test:
   
   **End-to-End Pipeline:**
   - Data → Features → Model → Predictions → Output
   - Verify all components work together
   - Check data flows correctly through pipeline
   
   ```python
   def test_full_pipeline():
       raw_data = load_raw_stdf_data()
       features = feature_engineering_pipeline(raw_data)
       predictions = model_pipeline.predict(features)
       api_response = format_response(predictions)
       
       assert api_response['status'] == 'success'
       assert len(api_response['predictions']) == len(raw_data)
   ```
   
   **Performance/Latency:**
   - Single prediction latency <100ms
   - Batch prediction throughput >1000 pred/sec
   - Memory usage within limits (<2GB)
   
   ```python
   def test_prediction_latency():
       import time
       data = generate_test_data(n=10000)
       
       start = time.time()
       predictions = pipeline.predict(data)
       elapsed_ms = (time.time() - start) * 1000
       
       throughput = 10000 / (elapsed_ms / 1000)
       assert throughput > 1000, f"Throughput too low: {throughput:.0f} pred/sec"
   ```
   
   **Error Handling:**
   - Graceful degradation on missing features
   - Informative error messages
   - Fallback to defaults when appropriate
   
   ```python
   def test_pipeline_handles_missing_features():
       data_with_missing = pd.DataFrame({'vdd': [1.2]})  # Missing idd, freq, temp
       
       try:
           pipeline.predict(data_with_missing)
           assert False, "Should raise error on missing features"
       except ValueError as e:
           assert 'missing columns' in str(e).lower()
   ```
   
   **Data Flow:**
   - Features match model's expected input
   - No feature leakage from training to test
   - Point-in-time correctness maintained
   
   **Monitoring Integration:**
   - Metrics logged correctly
   - Alerts trigger on anomalies
   - Dashboards update in real-time


7. MODEL VALIDATION STRATEGIES
   ----------------------------
   
   Offline Validation:
   
   **Holdout Set:**
   - Reserve 20% data for final validation
   - Never used for training or hyperparameter tuning
   - Simulates production performance
   
   **Cross-Validation:**
   - K-fold (k=5 or 10)
   - Time-series split (for temporal data)
   - Stratified (preserve class distribution)
   
   **Temporal Validation:**
   - Train on past data, test on future data
   - Critical for time-series, churn prediction
   - Example: Train on 2023, validate on 2024
   
   Online Validation (Production):
   
   **A/B Testing:**
   - New model (B) vs production model (A)
   - Split traffic 50/50 or 90/10 (conservative)
   - Measure business metrics (revenue, engagement)
   - Statistical significance test (t-test, chi-square)
   
   **Shadow Mode:**
   - New model runs alongside production
   - Predictions logged but not used
   - Compare to production model
   - Safe way to validate before go-live
   
   **Canary Deployment:**
   - Deploy to 5% of traffic first
   - Monitor closely for errors, degradation
   - Gradual rollout: 5% → 20% → 50% → 100%
   - Rollback if issues detected
   
   Validation Metrics:
   
   **Classification:**
   - Accuracy, Precision, Recall, F1
   - AUC-ROC, AUC-PR
   - Confusion matrix
   - Calibration (predicted probs match actual rates)
   
   **Regression:**
   - MAE, RMSE, MAPE
   - R² score
   - Residual plots (check for patterns)
   
   **Business Metrics:**
   - Revenue impact ($)
   - Customer satisfaction (NPS)
   - Cost reduction (%)
   - Time savings (hours)


8. TEST COVERAGE AND CI/CD
   ------------------------
   
   Code Coverage:
   - Target: >80% line coverage (pytest-cov)
   - Critical paths: 100% coverage
   - Generate reports: coverage html, coverage xml
   - Track over time (coverage should increase)
   
   ```bash
   pytest --cov=src --cov-report=html --cov-report=term
   ```
   
   Continuous Integration:
   - Run all tests on every commit
   - Fast tests (<5 min) in pre-commit
   - Full test suite in CI pipeline
   - Block merge if tests fail
   
   CI Pipeline Example:
   ```yaml
   # .github/workflows/test.yml
   name: ML Tests
   on: [push, pull_request]
   
   jobs:
     test:
       runs-on: ubuntu-latest
       steps:
         - uses: actions/checkout@v2
         - name: Set up Python
           uses: actions/setup-python@v2
           with:
             python-version: 3.9
         - name: Install dependencies
           run: pip install -r requirements.txt
         - name: Run unit tests
           run: pytest tests/unit --cov=src
         - name: Run integration tests
           run: pytest tests/integration
         - name: Upload coverage
           uses: codecov/codecov-action@v2
   ```
   
   Test Organization:
   ```
   tests/
   ├── unit/                  # Fast, isolated tests
   │   ├── test_data.py       # Data validation
   │   ├── test_features.py   # Feature engineering
   │   └── test_models.py     # Model components
   ├── integration/           # Slower, end-to-end tests
   │   ├── test_pipeline.py   # Full ML pipeline
   │   └── test_api.py        # API integration
   ├── performance/           # Load and stress tests
   │   └── test_latency.py
   └── fixtures/              # Shared test data
       └── sample_data.py
   ```


9. COMMON PITFALLS AND SOLUTIONS
   ------------------------------
   
   Pitfall 1: Data Leakage in Tests
   Problem: Test data contains information from training data
   Example: Using future data in features (violates point-in-time)
   Solution: Strict train/test split, temporal validation
   
   Pitfall 2: Overfitting to Test Set
   Problem: Tuning hyperparameters based on test set performance
   Example: Running 100 experiments, reporting best test accuracy
   Solution: Use validation set for tuning, holdout for final eval
   
   Pitfall 3: Ignoring Training-Serving Skew
   Problem: Features computed differently in training vs serving
   Example: Training uses SQL, serving uses Python (different results)
   Solution: Feature store ensures consistency
   
   Pitfall 4: Brittle Tests (Flaky)
   Problem: Tests pass/fail randomly
   Example: Tests depend on random initialization without seed
   Solution: Set random seeds, use deterministic algorithms
   
   Pitfall 5: Testing Implementation Not Interface
   Problem: Tests tied to internal implementation details
   Example: Testing internal variables instead of outputs
   Solution: Test public API and outputs, not internals
   
   Pitfall 6: Insufficient Edge Case Coverage
   Problem: Tests only cover happy path
   Example: Not testing nulls, zeros, empty inputs
   Solution: Property-based testing, explicit edge case tests
   
   Pitfall 7: No Performance Testing
   Problem: Model works but too slow for production
   Example: Prediction latency 5 seconds (SLA is 100ms)
   Solution: Integration tests include latency benchmarks
   
   Pitfall 8: Ignoring Model Degradation
   Problem: Model accuracy slowly decreases over time
   Example: Data drift causes 90% → 80% accuracy over 6 months
   Solution: Continuous monitoring, automated retraining


10. BEST PRACTICES
    ---------------
    
    Test-Driven Development (TDD):
    1. Write test first (it fails)
    2. Implement minimal code to pass test
    3. Refactor while keeping tests passing
    4. Repeat
    
    Benefits:
    - Forces thinking about requirements
    - Ensures testable code
    - High test coverage by design
    - Confidence in refactoring
    
    Testing Checklist:
    □ Data validation tests (schema, quality)
    □ Feature engineering tests (correctness, edge cases)
    □ Model tests (training, prediction, performance)
    □ Integration tests (end-to-end pipeline)
    □ Performance tests (latency, throughput)
    □ Property-based tests (invariants)
    □ Error handling tests (graceful degradation)
    □ Reproducibility tests (same seed → same results)
    □ CI/CD integration (auto-run on commit)
    □ Coverage tracking (>80% target)
    
    Test Quality:
    - Fast: Unit tests <10ms, integration tests <1s
    - Isolated: No dependencies on external services
    - Repeatable: Deterministic, same results every run
    - Self-checking: Automatic assertions (no manual inspection)
    - Timely: Written alongside code, not after
    
    Documentation:
    - Docstrings explain what test validates
    - Comments explain WHY, not what
    - README with how to run tests
    - Examples of good/bad inputs


11. PRODUCTION CHECKLIST
    ---------------------
    
    Before Deploying ML Model:
    
    Data Validation:
    □ Schema tests pass
    □ Data quality tests pass
    □ No data leakage detected
    □ Point-in-time correctness verified
    
    Model Validation:
    □ Accuracy meets threshold (>85%)
    □ No catastrophic errors (predictions <10x reality)
    □ Fairness tests pass (no demographic bias)
    □ Robustness verified (adversarial examples)
    □ Calibration acceptable (predicted probs match actual)
    
    Integration:
    □ End-to-end tests pass
    □ Latency within SLA (<100ms p99)
    □ Throughput adequate (>1000 pred/sec)
    □ Error handling graceful
    □ Monitoring integrated
    
    Deployment:
    □ A/B test configured
    □ Shadow mode validated
    □ Rollback plan documented
    □ Alerts configured
    □ Runbooks updated
    
    Post-Deployment:
    □ Monitor accuracy daily
    □ Track data drift
    □ Alert on performance degradation
    □ Automated retraining pipeline ready


12. NEXT STEPS
    -----------
    
    After ML Testing, continue with:
    
    126_Continuous_Training_Pipelines.ipynb:
    - Automated retraining when drift detected
    - Testing in retraining pipeline
    - Validation gates before deployment
    
    127_Model_Serving_Patterns.ipynb:
    - Testing serving infrastructure
    - Load testing API endpoints
    - Integration with monitoring
    
    128_ML_Monitoring.ipynb:
    - Testing monitoring systems
    - Validating alert logic
    - Dashboard functionality tests


================================================================================
FINAL SUMMARY
================================================================================

ML testing goes beyond traditional software testing to address unique
challenges: non-determinism, data dependencies, gradual degradation, and
the interplay between code, data, and models.

Key Principles:
1. Test data, features, and models (not just code)
2. Use statistical assertions (distributions, not exact values)
3. Property-based testing finds edge cases
4. Integration tests validate entire pipeline
5. Continuous validation in production (A/B, shadow mode)
6. Test coverage >80%, critical paths 100%
7. CI/CD integration prevents regressions

Critical Tests:
- Data schema and quality
- Feature engineering correctness
- Model performance baselines
- End-to-end pipeline integration
- Latency and throughput SLAs
- Reproducibility and determinism

Testing Pyramid: 70% unit, 20% integration, 10% system tests.

Remember: In ML, no test failure is often WORSE than a loud failure.
Silent accuracy degradation can cost millions before detection.

================================================================================
"""

print(takeaways)

## 8. Key Takeaways - ML Testing Mastery

Comprehensive reference guide for production ML testing strategies.

## 6. Integration Testing - End-to-End ML Pipeline

**What's happening:** Testing complete ML workflow from data → features → model → predictions.

**Key points:**
- **Pipeline tests**: Verify all components work together
- **Performance SLA**: Check latency, throughput under load
- **Data flow**: Input data → transformed correctly → valid predictions
- **Error handling**: System degrades gracefully on bad inputs

**Production scenario:** Before deploying, run full integration test simulating production traffic.

## 5. Model Testing - Training and Prediction Validation

**What's happening:** Testing model behavior, training stability, and prediction quality.

**Key points:**
- **Training tests**: Model trains without errors, converges, learns patterns
- **Prediction tests**: Output shapes, probability ranges, label consistency
- **Performance tests**: Accuracy baselines, no random guessing
- **Reproducibility**: Same seed → same results

**Why critical:** Models can train successfully but still be fundamentally broken (predicting all zeros, not learning).

## 4. Property-Based Testing - Invariants and Edge Cases

**What's happening:** Using Hypothesis library to generate random test cases and verify properties.

**Key points:**
- **Property testing**: Define invariants that should always hold
- **Automatic test generation**: Hypothesis generates 100+ test cases
- **Edge case discovery**: Find bugs you didn't think to test
- **Statistical properties**: Verify distributions, ranges, relationships

**Production value:** Catch obscure bugs that manual test cases miss (wafer edge cases, numerical precision).

## 3. Unit Testing Feature Engineering - Transformation Logic

**What's happening:** Testing feature engineering functions for correctness and edge cases.

**Key points:**
- **Deterministic outputs**: Same input → same output (reproducibility)
- **Edge cases**: Handle zeros, negatives, infinities, nulls
- **Mathematical correctness**: Verify formulas (power = vdd * idd)
- **Performance**: Test computation time for large datasets

**Why this matters:** Incorrect feature engineering is the #1 cause of training-serving skew.

## 2. Unit Testing Data Pipelines - Schema Validation

**What's happening:** Testing data ingestion and schema validation for STDF test data.

**Key points:**
- **Schema tests**: Data types, column names, required fields
- **Boundary tests**: Valid ranges for parametric values
- **Null handling**: Missing data detection and policies
- **Fail fast**: Catch data issues early (before model training)

**Post-silicon context:** STDF files must conform to IEEE 1505 standard with strict parametric ranges.